### Step 1: Import necessary libraries

In [ ]:
import numpy as np
import torch
import random
from src.dataset.eeg_dataset import EEGDataset
from pathlib import Path
import pandas as pd
import os
from src.utils import Utils

### Step 2: Set random seeds

In [ ]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.mps.manual_seed(random_seed)

In [ ]:

window_sizes = [128, 2 * 128, 5 * 128, 10 * 128, 30 * 128, 60*128]
step_sizes = [128, 2 * 128, 5 * 128, 10 * 128, 30 * 128, 60*128]

window_size = window_sizes[3]
step_size = step_sizes[3]
preprocessing = True
feature_selection = False
depth_of_anesthesia = True
for_majority = int(window_size / 2)
strategy = "stftEEG" 
sampling_rate = 128
training_ids=[1, 2, 3, 4, 6, 7, 12,]

#stft_window_size=1024
#stft_hop=256
stft_window_size=64
stft_hop=16

base_path = Path.cwd()
data_path = base_path / "data"

In [ ]:
print(
    f"You are using {strategy} with a window size of {window_size} and step size of {step_size} to predict Depth of Anesthesia"
)

In [ ]:
end_path = f"{strategy}_{window_size}_{step_size}_{stft_window_size}_{stft_hop}.h5"
training_data_path = data_path / f"training_data_{end_path}"
validation_data_path = data_path / f"validation_data_{end_path}"
test_data_path = data_path / f"test_data_{end_path}"

print(training_data_path)

if os.path.exists(training_data_path) and os.path.exists(validation_data_path) and os.path.exists(test_data_path):
    training_data = pd.read_hdf(data_path / f"training_data_{end_path}", key='eeg_window')
    test_data = pd.read_hdf(data_path / f"test_data_{end_path}", key='eeg_window')
    validation_data = pd.read_hdf(data_path / f"validation_data_{end_path}", key='eeg_window')
    
else:
    sleep_data = EEGDataset(
        data_dir=base_path / "EEG_data",
        training_ids=training_ids,  
        validation_ids=[11, 10, 9],
        testing_ids=[8, 5],
        window_size=window_size,
        sampling_rate=sampling_rate,
        step_size=step_size,
        majority_voting=True,
        for_majority=for_majority,
        preprocessing=preprocessing,
        inference_mode=False,
        depth_of_anesthesia=depth_of_anesthesia,
        strategy=strategy,
        stft_hop=stft_hop,
        stft_window_size=stft_window_size
    )
    
    training_data = sleep_data.train_df
    test_data = sleep_data.test_df
    validation_data = sleep_data.val_df
    training_data.to_hdf(
        data_path / f"training_data_{end_path}", key='eeg_window', index=False
    )
    test_data.to_hdf(data_path / f"test_data_{end_path}", key='eeg_window',index=False)
    validation_data.to_hdf(
        data_path / f"validation_data_{end_path}", key='eeg_window',index=False
    )

In [ ]:
for column in training_data.columns:
    print(column)
print(training_data.shape)
test_data

### Step 3: Datasets get prepared and preprocessed for the different binary classification targets

In [ ]:
utils = Utils(
    for_majority=for_majority,
    window_size=window_size,
    step_size=step_size,
    random_seed=random_seed,
    preprocessing=preprocessing,
    sampling_rate=sampling_rate,
    results_validation_csv_path=base_path
    / "doA_classification"
    / "ml_models"
    / "validation_results_df.csv",
    results_test_csv_path=base_path
    / "doA_classification"
    / "ml_models"
    / "test_results_df.csv",
    model_dir=base_path / "doA_classification" / "ml_models",
    stft_window_size=stft_window_size,
    stft_hop=stft_hop
)

exclude_columns = ["Start", "End", "sleep"]
labels = ["sleep"]
if depth_of_anesthesia:
    exclude_columns.extend(["cr", "sspl", "burst_suppression"])
    labels_to_process = ["sleep", "cr", "sspl", "burst_suppression"]
    labels.extend(["cr", "sspl", "burst_suppression"])
else:
    labels_to_process = ["sleep"]

# Define features (excluding the necessary columns)
features = training_data.drop(columns=exclude_columns, errors="ignore").columns

In [ ]:
print(np.array(test_data['stft_window'].tolist()).shape)

In [ ]:
# Create a new dictionary to store preprocessed data
preprocessed_data_dict = {}


for label in labels_to_process:
    print(f"Processing {label}...")
    # Preprocess data
    (
        X,
        y,
        X_val,
        y_val,
        X_test,
        y_test,
        train_loader_nn,
        val_loader_nn,
        test_loader_nn,
        input_size,
    ) = utils.preprocess_data(
        X=np.expand_dims(np.array(training_data['stft_window'].tolist()), axis=1),
        y=training_data[label],
        X_val=np.expand_dims(np.array(validation_data['stft_window'].tolist()), axis=1),
        y_val=validation_data[label],
        X_test=np.expand_dims(np.array(test_data['stft_window'].tolist()), axis=1),
        y_test=test_data[label],
        batch_size=512,   #1024 until 3840 ->512, 60s -> 256
        device="mps",
        strategy=strategy,
        classification_type=label,
        scaling_nn=False,
        imbalanced=True,
    )

    preprocessed_data_dict[label] = {
        "X": X,
        "y": y,
        "X_val": X_val,
        "y_val": y_val,
        "X_test": X_test,
        "y_test": y_test,
        "train_loader_nn": train_loader_nn,
        "val_loader_nn": val_loader_nn,
        "test_loader_nn": test_loader_nn,
        "input_size": input_size,
    }

print("Processing completed for all labels.")

In [ ]:
print(X_test[0].shape)
input_size = X_test[0].shape[1]

### Step 4: Define ML-Models 

In [ ]:
from src.models.LSTM import LSTMModel
from src.models.UCR import SimpleModel, FCN, UCRResNet
from src.models.Transformer import TSTransformerEncoderClassiregressor
from torchvision.models import resnet18
import torch.nn as nn

model = resnet18().to('mps')
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False).to('mps')
model.fc = nn.Linear(model.fc.in_features, 1).to('mps')

models = {
    "UCRResNet": UCRResNet(
        input_shape=input_size,
        n_feature_maps=64,
        nb_classes=1
    ).to('mps'),
    "LSTMModel": LSTMModel(input_size=input_size, 
                      hidden_size=256, 
                      num_layers=2, 
                      output_size=1, 
                      dropout_rnn=0.3, 
                      dropout=0.1,
                      bidirectional=False).to('mps'),
    "FCN": FCN(
        input_shape=input_size,
        num_classes=1
    ).to('mps'),
    "ResNet": model
}

### Step 5: Train the Models for the different binary classification tasks

In [ ]:
for label in ["sleep", "cr", "sspl", "burst_suppression"]:
    for model_name, model in models.items():
        utils.train_and_evaluate_model(
            name=model_name,
            model=model,
            train_loader_nn=preprocessed_data_dict[label]["train_loader_nn"],
            val_loader_nn=preprocessed_data_dict[label]["val_loader_nn"],
            classification_type=label,
            strategy=strategy,
            learning_rate=1e-4,
            patience=5)

### Step 6: Get Test Results:

In [ ]:

get_test_results = True 
if get_test_results:
    for label in ["sleep", "cr", "sspl","burst_suppression"]: 

        for model_name, model in models.items():

            test_results = utils.evaluate_model_on_test(
                model_name=model_name,
                model=model,
                X_test=preprocessed_data_dict[label]["X_test"],
                y_test=preprocessed_data_dict[label]["y_test"],
                test_loader=preprocessed_data_dict[label]["test_loader_nn"],
                classification_type=label,
                strategy=strategy,
            )